# Query PDF Tutorial

In this tutorial, we demonstrate how to load a PDF and query it.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB


In [1]:
%pip install --quiet "evadb[document,notebook]"
%pip install reportlab
import evadb
cursor = evadb.connect().cursor()
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.1/530.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 1.06MB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


### Download PDFs

In [43]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

from bs4 import BeautifulSoup
from urllib.request import urlopen

# URL of the web page
url = "https://buzzdb-docs.readthedocs.io/part1/lab1.html"
#url = input("Enter the URL of the web page: ")
# Fetch the web page content
page = urlopen(url)
html = page.read().decode("utf-8")

# Parse the HTML content with Beautiful Soup
soup = BeautifulSoup(html, "html.parser")

# Find and extract paragraphs
text_list = [p.get_text() for p in soup.find_all("p")]
print(len(text_list))


83


In [44]:
def merge_strings(text_list):
    result_list = []
    current_string = ""
    word_count = 0

    for text in text_list:
        words = text.split()
        for word in words:
            current_string += word + " "
            word_count += 1
            if word_count >= 300:
                result_list.append(current_string.strip())  # 去除末尾的空格
                current_string = ""
                word_count = 0

    # check all strings are appended
    if current_string:
        result_list.append(current_string.strip())

    return result_list

# merge all strings
merged_list = merge_strings(text_list)

# print merged strings
for i, merged_text in enumerate(merged_list):
    print(f"String {i + 1}:", merged_text)

String 1: Due: 02/01/2023 11:59 PM EST The goal of this assignment is to help you brush up your C++ programming skills, and exercise your skills in Data Structure and Algorithm Design. In this assignment, you are to develop a word locator program written in C++, which will allow a user to check if a specified (re)occurrence of a specified query word appears in the input text file. CS4420 students can use the unordered map data structure from Stl. CS6422 students are required to use a Tree-based data structure. The code submissions will be checked after submission to ensure that this is followed. NOTE : Before jumping into the assignment, make sure to setup the development environment following the instructions given here. For this programming assignment, you are provided with a skeleton code which you are expected to complete and submit. You are expected to fill the execute function of CommandExecutor class, present in these files: buzzdb/src/tutorial/tutorial.cc and buzzdb/src/include

In [70]:
# compute the number of word of the website
num = 0
for s in merged_list:
    words = s.split()
    num = num+len(words)
print("Origin website has :", num, "words.")

Origin website has : 1813 words.


In [52]:

# Create a PDF document
pdf_file = "output.pdf"
document = SimpleDocTemplate(pdf_file, pagesize=letter)

# Create a list of flowable elements (e.g., paragraphs and spacers)
elements = []

# Define the style for the paragraphs
styles = getSampleStyleSheet()
style = styles["Normal"]

# Create a paragraph for each item in the list and add a spacer
for text in merged_list:
    paragraph = Paragraph(text, style)
    elements.append(paragraph)
    elements.append(Spacer(1, 12))  # Add an empty line (adjust the second parameter for spacing)

# Build the PDF document
document.build(elements)

print(f"List of paragraphs with empty lines has been successfully converted to a PDF: {pdf_file}")


List of paragraphs with empty lines has been successfully converted to a PDF: output.pdf


In [53]:
cursor.query("DROP TABLE IF EXISTS MyPDFs").df()
cursor.query("LOAD PDF 'output.pdf' INTO MyPDFs").df()

,0
0,Number of loaded PDF: 1


### Load PDFs

### Retrieve Text from Loaded PDFs

In [54]:
cursor.query("SELECT * FROM MyPDFs").df()

,mypdfs._row_id,mypdfs.name,mypdfs.page,mypdfs.paragraph,mypdfs.data
0,1,output.pdf,1,1,Due: 02/01/2023 11:59 PM EST The goal of this ...
1,1,output.pdf,1,2,"as if there is an intermediate ""new"" command (..."
2,1,output.pdf,1,3,the definition above. For example ra#s and rat...
3,1,output.pdf,2,1,bird[[ And snipped off her nose! The following...
4,1,output.pdf,2,2,21 > locate pie 3 No matching entry > locate p...
5,1,output.pdf,2,3,information. CMakeLists.txt: CMake file for bu...
6,1,output.pdf,2,4,in the system might be used up. Valgrind can b...
7,1,output.pdf,3,1,your choice of the data structure that you imp...
8,1,output.pdf,3,2,score less than 100 we will award partial poin...


In [23]:
# this is just a simple test of the evadb content
text = cursor.query("""
    SELECT data
    FROM MyPDFs
    WHERE page = 1 AND paragraph = 3
""").df()
s = text['mypdfs.data']
x_list = s.tolist()
print(x_list[0])

CS4420 students can use the unordered map data structure from Stl. CS6422 students are required touse a Tree-based data structure. The code submissions will be checked after submission to ensure thatthis is followed.


### Create UDFs for Text Summarization

In [25]:
cursor.query("""
    CREATE FUNCTION IF NOT EXISTS TextSummarizer
    TYPE HuggingFace
    TASK 'summarization'
    MODEL 'facebook/bart-large-cnn'
""").df()

Your max_length is set to 142, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


,0
0,Function TextSummarizer added to the database.


In [66]:
string = """
    SELECT data, TextSummarizer(data)
    FROM MyPDFs
"""
result = cursor.query(string).df()

Your max_length is set to 142, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 142, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


In [71]:
s = result['textsummarizer.summary_text']
x_list = s.tolist()
num = 0
for s in x_list:
    words = s.split()
    num = num+len(words)
merged_string = " ".join(x_list)

# output the result length and Summarization
print("Total word:", num)
print(merged_string)

Total word: 478
This assignment is to help you brush up your C++programming skills, and exercise your skills in Data Structure and Algorithm Design. In this assignment,you are to develop a word locator program written in C++, which will allow a user to check if a specified(re)occurrence of a specified query word appears in the input text file. The locate command is case insensitive, i.e. to match the word in the locate command with a word in a load file you should use a case-insensitive string comparison method. The syntax of the locatecommand is "locate". The parameter will have a whitespace before and after it, and should be an integer greater than 0. If an incorrectload command is entered, such as "load" then your data structure should not be reset. For example ra#s and rats! are invalid word parameters. All the command keywords are case insensitive, so "LoCATe sing 2" is a valid command,and should be treated as "locate sing 2". The following is a sample run: Sample Run > load data/